# Note
- Succeeded in local 'Llama3-8b' after set 'memory=False'
- Switch to Groq for faster inference:
    - pip install langchain-groq
    - Search 'https://console.groq.com/playground' for models
          - 'llama3-8b-8192'
    - Set up api key in 'https://console.groq.com/keys'
    - Set up current directory '.env'
    - Call 'ChatGroq()'
- Groq with 'llama3-70b-8192' successfully
- If to avoid rate limited exception
    - Token per minute limit
    - Set 'max_rpm=2' in 'Crew()'

# L4: Tools for a Customer Outreach Campaign

In this lesson, you will learn more about Tools. You'll focus on three key elements of Tools:
- Versatility
- Fault Tolerance
- Caching

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [20]:
import sys
!{sys.executable} -m pip install langchain-groq

In [21]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM
- [Serper](https://serper.dev)

In [22]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [23]:
import os
from utils import get_openai_api_key, pretty_print_result
from utils import get_serper_api_key

openai_api_key = get_openai_api_key()
#os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

In [24]:
"""
import os
from dotenv import load_dotenv

load_dotenv()

OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
OPENAI_API_BASE = os.getenv('OPENAI_API_BASE')
OPENAI_MODEL_NAME = os.getenv('OPENAI_MODEL_NAME')
"""

"\nimport os\nfrom dotenv import load_dotenv\n\nload_dotenv()\n\nOPENAI_API_KEY = os.getenv('OPENAI_API_KEY')\nOPENAI_API_BASE = os.getenv('OPENAI_API_BASE')\nOPENAI_MODEL_NAME = os.getenv('OPENAI_MODEL_NAME')\n"

In [25]:
from crewai import Agent, Task, Crew
from langchain_openai import ChatOpenAI

"""
os.environ["OPENAI_API_KEY"] = "NA"

# Select the local model
llm = ChatOpenAI(
    model = "crewai-llama3-8b",
    base_url = "http://localhost:11434/v1")
"""

'\nos.environ["OPENAI_API_KEY"] = "NA"\n\n# Select the local model\nllm = ChatOpenAI(\n    model = "crewai-llama3-8b",\n    base_url = "http://localhost:11434/v1")\n'

In [26]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv

load_dotenv()

llm = ChatGroq(
    api_key = os.getenv('GROQ_API_KEY'),
    #model = "llama3-8b-8192"
    model = "llama3-70b-8192"
)


## Creating Agents

In [27]:
sales_rep_agent = Agent(
    role="Sales Representative",
    goal="Identify high-value leads that match "
         "our ideal customer profile",
    backstory=(
        "As a part of the dynamic sales team at CrewAI, "
        "your mission is to scour "
        "the digital landscape for potential leads. "
        "Armed with cutting-edge tools "
        "and a strategic mindset, you analyze data, "
        "trends, and interactions to "
        "unearth opportunities that others might overlook. "
        "Your work is crucial in paving the way "
        "for meaningful engagements and driving the company's growth."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

In [28]:
lead_sales_rep_agent = Agent(
    role="Lead Sales Representative",
    goal="Nurture leads with personalized, compelling communications",
    backstory=(
        "Within the vibrant ecosystem of CrewAI's sales department, "
        "you stand out as the bridge between potential clients "
        "and the solutions they need."
        "By creating engaging, personalized messages, "
        "you not only inform leads about our offerings "
        "but also make them feel seen and heard."
        "Your role is pivotal in converting interest "
        "into action, guiding leads through the journey "
        "from curiosity to commitment."
    ),
    allow_delegation=False,
    verbose=True,
    llm=llm
)

## Creating Tools

### crewAI Tools

In [29]:
from crewai_tools import DirectoryReadTool, \
                         FileReadTool, \
                         SerperDevTool

In [30]:
directory_read_tool = DirectoryReadTool(directory='./instructions')
file_read_tool = FileReadTool()
search_tool = SerperDevTool()

### Custom Tool
- Create a custom tool using crewAi's [BaseTool](https://docs.crewai.com/core-concepts/Tools/#subclassing-basetool) class

In [31]:
from crewai_tools import BaseTool

- Every Tool needs to have a `name` and a `description`.
- For simplicity and classroom purposes, `SentimentAnalysisTool` will return `positive` for every text.
- When running locally, you can customize the code with your logic in the `_run` function.

In [32]:
class SentimentAnalysisTool(BaseTool):
    name: str ="Sentiment Analysis Tool"
    description: str = ("Analyzes the sentiment of text "
         "to ensure positive and engaging communication.")
    
    def _run(self, text: str) -> str:
        # Your custom code tool goes here
        return "positive"

In [33]:
sentiment_analysis_tool = SentimentAnalysisTool()

## Creating Tasks

- The Lead Profiling Task is using crewAI Tools.

In [34]:
lead_profiling_task = Task(
    description=(
        "Conduct an in-depth analysis of {lead_name}, "
        "a company in the {industry} sector "
        "that recently showed interest in our solutions. "
        "Utilize all available data sources "
        "to compile a detailed profile, "
        "focusing on key decision-makers, recent business "
        "developments, and potential needs "
        "that align with our offerings. "
        "This task is crucial for tailoring "
        "our engagement strategy effectively.\n"
        "Don't make assumptions and "
        "only use information you absolutely sure about."
    ),
    expected_output=(
        "A comprehensive report on {lead_name}, "
        "including company background, "
        "key personnel, recent milestones, and identified needs. "
        "Highlight potential areas where "
        "our solutions can provide value, "
        "and suggest personalized engagement strategies."
    ),
    tools=[directory_read_tool, file_read_tool, search_tool],
    agent=sales_rep_agent,
)

- The Personalized Outreach Task is using your custom Tool `SentimentAnalysisTool`, as well as crewAI's `SerperDevTool` (search_tool).

In [35]:
personalized_outreach_task = Task(
    description=(
        "Using the insights gathered from "
        "the lead profiling report on {lead_name}, "
        "craft a personalized outreach campaign "
        "aimed at {key_decision_maker}, "
        "the {position} of {lead_name}. "
        "The campaign should address their recent {milestone} "
        "and how our solutions can support their goals. "
        "Your communication must resonate "
        "with {lead_name}'s company culture and values, "
        "demonstrating a deep understanding of "
        "their business and needs.\n"
        "Don't make assumptions and only "
        "use information you absolutely sure about."
    ),
    expected_output=(
        "A series of personalized email drafts "
        "tailored to {lead_name}, "
        "specifically targeting {key_decision_maker}."
        "Each draft should include "
        "a compelling narrative that connects our solutions "
        "with their recent achievements and future goals. "
        "Ensure the tone is engaging, professional, "
        "and aligned with {lead_name}'s corporate identity."
    ),
    tools=[sentiment_analysis_tool, search_tool],
    agent=lead_sales_rep_agent,
)

## Creating the Crew

In [37]:
crew = Crew(
    agents=[sales_rep_agent, 
            lead_sales_rep_agent],
    
    tasks=[lead_profiling_task, 
           personalized_outreach_task],
	
    verbose=2,
	#memory=True,
    memory=False
    #max_rpm=2
)

2024-06-16 14:47:41,637 - 8472759296 - __init__.py-__init__:518 - WARNING: Overriding of current TracerProvider is not allowed


## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [38]:
inputs = {
    "lead_name": "DeepLearningAI",
    "industry": "Online Learning Platform",
    "key_decision_maker": "Andrew Ng",
    "position": "CEO",
    "milestone": "product launch"
}

result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Sales Representative
 [INFO]: == Starting Task: Conduct an in-depth analysis of DeepLearningAI, a company in the Online Learning Platform sector that recently showed interest in our solutions. Utilize all available data sources to compile a detailed profile, focusing on key decision-makers, recent business developments, and potential needs that align with our offerings. This task is crucial for tailoring our engagement strategy effectively.
Don't make assumptions and only use information you absolutely sure about.


> Entering new CrewAgentExecutor chain...
Thought: I need to gather information about DeepLearningAI, let's start by listing the files in the directory to see if there's any information about the company.

Action: List files in directory
Action Input: {} 

File paths: 
-./instructions/enterprise_solutions_framework.md
- ./instructions/tech_startups_outreach.md
- ./instructions/small_business_engagement.md

Thought: The files listed don't seem to 

- Display the final result as Markdown.

In [39]:
from IPython.display import Markdown
Markdown(result)

Here are three personalized email drafts tailored to DeepLearningAI, specifically targeting Andrew Ng:

**Email Draft 1:**
Dear Andrew,

Congratulations on the recent launch of your new course, 'Building Your Own Database Agent' in collaboration with Microsoft. At [Your Company], we're excited about the innovative work you're doing at DeepLearningAI. Our solutions are designed with AI-focused organizations like yours in mind, offering advanced data analysis tools to enhance your online learning platform and improve efficiency. We understand the importance of scalability and flexibility in your mission to grow and connect the global AI community. Our solutions can be tailored to support your current and future needs, providing a seamless learning experience for your community. Let's explore how we can support DeepLearningAI's growth and mission together.

Best,
[Your Name]

**Email Draft 2:**
Dear Andrew,

We're thrilled to see DeepLearningAI's commitment to advancing AI education and training. Your recent collaboration with Microsoft on the 'Building Your Own Database Agent' course is a testament to your organization's innovative spirit. At [Your Company], we believe our solutions can help take your online learning platform to the next level. Our advanced data analysis tools can help you streamline data analysis, freeing up more time for what matters most - growing and connecting the global AI community. We'd love to explore how our solutions can support your mission and help you achieve your goals.

Best,
[Your Name]

**Email Draft 3:**
Dear Andrew,

As a pioneer in AI education, DeepLearningAI is shaping the future of the industry. We're excited to see how your recent course launch with Microsoft is simplifying data analysis for your community. At [Your Company], we're passionate about empowering AI-focused organizations like yours with cutting-edge solutions. Our advanced data analysis tools can help you unlock new insights, enhance your online learning platform, and drive engagement within your community. We believe our solutions can be a game-changer for DeepLearningAI, and we'd love to discuss how we can support your mission to grow and connect the global AI community.

Best,
[Your Name]